# Praca domowa nr 2
#### Bartosz Sawicki

In [ ]:
import category_encoders as ce
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

## Wczytanie zbioru danych

In [ ]:
url = 'https://www.dropbox.com/s/360xhh2d9lnaek3/allegro-api-transactions.csv?dl=1'

input_df = pd.read_csv(url)
input_df.head()

In [ ]:
input_df.shape

In [ ]:
input_df.info()

In [ ]:
input_df['date'] = pd.to_datetime(input_df.date, format="%Y-%m-%d %H:%M:%S")

In [ ]:
df = input_df.drop(['lp', 'item_id', 'categories', 'date'], axis='columns')

Usuwamy niepotrzebne kolumny.
Usuwamy kategorie i datę dla uproszczenia regresji.

## Podział na zbiór treningowy i testowy

In [ ]:
columns = df.columns.drop('price')

X_train, X_test, y_train, y_test = train_test_split(
    df[columns], 
    df['price'], 
    test_size=0.33, random_state=42)

# 1. Kodowanie zmiennych kategorycznych

## Target encoding `it_location`

In [ ]:
target_encoder = ce.TargetEncoder(cols=['it_location'])
target_encoder.fit_transform(X_train,y_train)

##  Kodowanie `main_category`
### One-Hot

In [ ]:
one_hot_encoder = ce.OneHotEncoder(cols=['main_category'])
one_hot = one_hot_encoder.fit_transform(X_train,y_train)

One Hot Encoder dla każdej kategorii tworzy nową kolumnę i wstawia do niej 1 gdy obserwacja należy do tej kategorii, 0 w przeciwnym przypadku.

### Hashing Difference Coding

In [ ]:
hashing_encoder = ce.HashingEncoder(cols=['main_category'])
hash_ = hashing_encoder.fit_transform(X_train,y_train)

Hashing encoding działa podobnie jak One Hot, ale przypisuje kategorie do kolumn na podstawie funkcji hashującej, której parametry można ustawić (w szczególności zbiór wartości), więc liczba wynikowych kolumn jest pod kontrolą programisty. Tym kodowaniem możemy kontrolować wymiary zakodowanego zbioru. Gdy ustawimy `n_components`=`len(df.columns)` otrzymamy kodowanie one-hot.

### Generalized Linear Mixed Model Encoder

In [ ]:
glmm_encoder = ce.GLMMEncoder(cols=['main_category'])
glmm = glmm_encoder.fit_transform(X_train,y_train)

Generalized Linear Mixed Model Encoder to samo tuningująca się odmaina Target Encodera. Dla każdej kategorii wylicza regularyzowaną różnicę średniej kategorii od średniej całego zbioru.

In [ ]:
labels = ['glmm', 'hash', 'one-hot']
size = [len(x.columns) for x in [glmm, hash_, one_hot]]

sns.barplot(x=labels, y=size, color='orange').set_title("Size of the transformed data-frame")
plt.show()

## Budowa modelu regresji liniowej

dla sprawdzenia jak różne kodowanie wpływa na skuteczność

### Musimy jeszcze zakodować `seller`

Użyjemy target encoder dlatego zmodyfikujemy już istniejący

In [ ]:
target_encoder = ce.TargetEncoder(cols=['it_location', 'seller'])
target_encoder.fit_transform(X_train,y_train)

In [ ]:
pipe_one_hot = Pipeline(
[
    ('transformer_target', target_encoder),
    ('transformer_one_hot', one_hot_encoder),
    ('linear-model', LinearRegression())
])

pipe_backward = Pipeline(
[
    ('transformer_target', target_encoder),
    ('transformer_hashing', hashing_encoder),
    ('linear-model', LinearRegression())
])

pipe_glmm = Pipeline(
[
    ('transformer_target', target_encoder),
    ('transformer_glmm', glmm_encoder),
    ('linear-model', LinearRegression())
])

pipes = [pipe_backward, pipe_glmm, pipe_one_hot]

In [ ]:
for pipe in pipes:
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    print(f' {pipe.steps[1][0]} RMSE : {np.sqrt(mean_squared_error(y_test, y_pred)):.3f}')

W tym przypadku najlepsze okazało się kodowanie one-hot, najgorsze natomiast kodowanie hashujące.

# 2. Uzupełnianie braków

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
input_df.info()

Aby skrócić czas obliczeń ograniczylem zbiór danych do 1000 obserwacji.

Poniżej dla `i`=1,...10 wykonano 10 razy imputację z parametrem `n_neighbors`=`i` z jedną brakującą kolumną (`it_seller_rating`).

In [ ]:
errors = {}

def remove_and_impute(n_neighbors):
    df2 = input_df[['price', 'it_seller_rating', 'it_quantity']].sample(1000, random_state=997).copy(deep=True).reset_index()
    
    removed = df2['it_seller_rating'].sample(frac=.1)
    df2.loc[removed.index,'it_seller_rating'] = np.nan
    
    imputer = KNNImputer(n_neighbors=n_neighbors, weights="uniform")
    imputed = imputer.fit_transform(df2)
    
    if errors.get(n_neighbors-1) is None:
        errors.update({n_neighbors-1 :[]})
    errors.get(n_neighbors-1).append(np.sqrt(mean_squared_error(imputed[removed.index,2], removed)))
    
    
for i in range(10):
    for j in range(10):
        remove_and_impute(i+1)
        
errors_df = pd.DataFrame(errors) # cols = n_neighbors-1
std1 = errors_df.describe().loc['std']

Następnie usunięto 2 kolumny (`it_seller_rating`, `it_location`) i powtórzono wcześniejszą procedurę

In [ ]:
errors_2 = {}

def remove_and_impute(n_neighbors):
    df2 = input_df[['price', 'it_seller_rating', 'it_quantity']].sample(1000, random_state=997).copy(deep=True).reset_index()
    
    removed = df2.sample(frac=.1)
    df2.loc[removed.index,['it_seller_rating', 'it_location']] = np.nan
     
    imputer = KNNImputer(n_neighbors=n_neighbors, weights="uniform")
    imputed = imputer.fit_transform(df2)
    
    if errors_2.get(n_neighbors-1) is None:
        errors_2.update({n_neighbors-1 :[]})
    errors_2.get(n_neighbors-1).append(np.sqrt(mean_squared_error(imputed[removed.index,2], removed['it_seller_rating'])))
    
    
for i in range(10):
    for j in range(10):
        remove_and_impute(i+1)


errors_2df = pd.DataFrame(errors_2)
std2 = errors_2df.describe().loc['std']

In [ ]:
fig, axes = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(15,5))
fig.suptitle('Porównanie imputacji KNN przy jednej i dwóch brakujących zmiennych')
axes[0].set_title('Brakuje `it_seller_rating`')
axes[1].set_title('Brakuje `it_seller_rating` oraz `it_location`')

ax = sns.boxplot(ax = axes[0], data = errors_df)
ax.set_xticklabels(labels = [i+1 for i in range(len(errors_df.columns))])
ax.set_xlabel('n_neighbors')
ax.set_ylabel('RMSE (imputed vs real values)')


ax3 = sns.boxplot(ax = axes[1], data = errors_2df)
ax3.set_xticklabels(labels = [i+1 for i in range(len(errors_2df.columns))])
ax3.set_xlabel('n_neighbors')
ax3.set_ylabel('RMSE (imputed vs real values)')


fig, axes = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(15,5))
fig.suptitle('Średnia i odchylenie standardowe RMSE imputacji w 10 próbach')
ax2 = sns.lineplot(ax=axes[0], data = pd.melt(errors_df),x = pd.melt(errors_df)['variable']+1, y = 'value', ci='sd')
ax4 = sns.lineplot(ax=axes[1], data = pd.melt(errors_2df),x = pd.melt(errors_df)['variable']+1, y = 'value', ci='sd')
ax2.set_xlabel('n_neighbors')
ax4.set_xlabel('n_neighbors')

Wyniki imputacji są podobne, niezależnie od tego czy opieramy się na jednej czy na dwóch kolumnach